## Imports and Options
- Place %%pycodestyle at the top of any cell to check python syntax

In [6]:
%load_ext pycodestyle_magic

###### import pandas as pd
import numpy as np
import shap
import pandas as pd
from pandasql import sqldf
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from getxgboostmodel.getxgboostmodel import get_xgboost_model
from randomizedgridsearch.randomizedgridsearch import RandomizedGridSearch
from transformers.transformers import *

# Lambdas


def q(x):
    return sqldf(x, globals())


The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


## Hello

In [7]:
# Standard scaler data preparation class


class StandardScalerTransform(BaseEstimator, TransformerMixin):
    def __init__(self, include_feature_i=[]):
        self.standard_scalers = {}

    def fit(self, X, y=None):
        for index in range(len(self.include_feature_i)):
            if self.include_feature_i[index] > 0.5:
                self.standard_scalers[index] = StandardScaler()
                self.standard_scalers[index].fit(X[:, index:index + 1])
        return self

    def transform(self, X, y=None):
        for index in range(len(self.include_feature_i)):
            if self.include_feature_i[index] > 0.5:
                X[:, index:index + 1] = self.standard_scalers[index].transform(
                    X[:, index:index + 1])
        return np.c_[X]


# Min-max scaler data preparation class


class MinMaxScalerTransform(BaseEstimator, TransformerMixin):
    def __init__(self, include_feature_i=[]):
        self.min_max_scalers = {}

    def fit(self, X, y=None):
        for index in range(len(self.include_feature_i)):
            if self.include_feature_i[index] > 0.5:
                self.min_max_scalers[index] = MinMaxScaler()
                self.min_max_scalers[index].fit(X[:, index:index + 1])
        return self

    def transform(self, X, y=None):
        for index in range(len(self.include_feature_i)):
            if self.include_feature_i[index] > 0.5:
                X[:, index:index + 1] = self.min_max_scalers[index].transform(
                    X[:, index:index + 1])
        return np.c_[X]


# Binarizer scaler data preparation class


class BinarizerTransform(BaseEstimator, TransformerMixin):
    def __init__(self, include_feature_i={}):
        self.thresholds = {}

    def fit(self, X, y=None):
        for index in range(len(self.include_feature_i)):
            if self.include_feature_i[index] == True:
                self.thresholds[index] = np.quantile(X[:, index:index + 1],
                                                 0.50)
        return self

    def transform(self, X, y=None):
        for index in range(len(self.include_feature_i)):
            if self.include_feature_i[index] == True:
                X[:, index:index +
                  1] = X[:, index:index + 1] > self.thresholds[index]
        return np.c_[X]


In [23]:
min_max_values

[None, None, None, None, True, None, None, None, None, None, None, None, None]

In [22]:
# Example dataset
boston_data = load_boston()

# Extract pandas dataframe and target
X = pd.DataFrame(boston_data['data']).copy().values
y = pd.DataFrame(boston_data['target']).copy().values

# Train/test split
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.20, random_state=42)
train_X, test_X = train_X, test_X
train_y, test_y = train_y.reshape(-1, 1), test_y.reshape(-1, 1)

# An okay model fit to the data
try:
    xgb_model
except:
    xgb_model = get_xgboost_model(train_X, train_y)
# linear_regression = LinearRegression(normalize=False)

# Pipeline
pipe = Pipeline([('standard_scaler', StandardScalerTransform()),
                 ('min_max_scaler', MinMaxScalerTransform()),
                 ('binarizer', BinarizerTransform()), 
                 ('model', xgb_model)])

# Find the number of features
num_features = train_X.shape[1]

# Testing with these indices
indices = list(range(num_features))

default_values = [None]*num_features

min_max_values = [None]*num_features
min_max_values[4] = True

binarizer_values = [False]*num_features
binarizer_values[3] = True
binarizer_values[12] = True

# Possible configurations [None, True, or False] - None means not decided yet
param_distributions = {
    'standard_scaler__custom_values': default_values,
    'min_max_scaler__custom_values': min_max_values,
    'binarizer__custom_values': binarizer_values
}

# Randomly search the space n_iter times
experiments_df = RandomizedGridSearchCV(
    n_experiments=100,
    pipe=pipe,
    param_distributions=param_distributions,
    train_X=train_X,
    train_y=train_y,
    test_X=test_X,
    test_y=test_y,
    scoring='neg_mean_squared_error',
    cv=2)

# Sort the scores
experiments_df.sort_values(by=['score'], ascending=False, inplace=True)

# Drop score
experiments_X_df = experiments_df.drop(['score'], axis=1)

# Get column names
X_column_names = experiments_X_df.columns

# Convert to numpy
experiments_X = experiments_X_df.values
experiments_y = experiments_df[['score']].values

# Create an XGBoost model tuned with the experiments data
xgb_experiments_model = get_xgboost_model(experiments_X, experiments_y)

# Fit the model
xgb_experiments_model.fit(experiments_X_df, experiments_y)

# Extract shap values
explainer = shap.TreeExplainer(xgb_experiments_model)
shap_values = explainer.shap_values(experiments_X_df)

# Shap as dataframe
pandas_shap_df = pd.DataFrame(shap_values, columns=X_column_names)
pandas_shap_df


Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Itera

standard_scaler__custom_values___0  standard_scaler__custom_values___1  \
0                                  0.0                                 0.0   
1                                  0.0                                 0.0   
2                                  0.0                                 0.0   
3                                  0.0                                 0.0   
4                                  0.0                                 0.0   
5                                  0.0                                 0.0   
6                                  0.0                                 0.0   
7                                  0.0                                 0.0   
8                                  0.0                                 0.0   
9                                  0.0                                 0.0   
10                                 0.0                                 0.0   
11                                 0.0                                 0.0   
12                                 0.0                                 0.0   
13                                 0.0                                 0.0   
14                                 0.0                                 0.0   
15                                 0.0                                 0.0   
16                                 0.0                                 0.0   
17                                 0.0                                 0.0   
18                                 0.0                                 0.0   
19                                 0.0                                 0.0   
20                                 0.0                                 0.0   
21                                 0.0                                 0.0   
22                                 0.0                                 0.0   
23                                 0.0                                 0.0   
24                                 0.0                                 0.0   
25                                 0.0                                 0.0   
26                                 0.0                                 0.0   
27                                 0.0                                 0.0   
28                                 0.0                                 0.0   
29                                 0.0                                 0.0   
..                                 ...                                 ...   
70                                 0.0                                 0.0   
71                                 0.0                                 0.0   
72                                 0.0                                 0.0   
73                                 0.0                                 0.0   
74                                 0.0                                 0.0   
75                                 0.0                                 0.0   
76                                 0.0                                 0.0   
77                                 0.0                                 0.0   
78                                 0.0                                 0.0   
79                                 0.0                                 0.0   
80                                 0.0                                 0.0   
81                                 0.0                                 0.0   
82                                 0.0                                 0.0   
83                                 0.0                                 0.0   
84                                 0.0                                 0.0   
85                                 0.0                                 0.0   
86                                 0.0                                 0.0   
87                                 0.0                                 0.0   
88                                 0.0                                 0.0   
89                        

In [3]:
xgb_experiments_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, xgb__colsample_bylevel=0.9,
       xgb__colsample_bytree=1.0, xgb__learning_rate=0.1, xgb__max_depth=1,
       xgb__n_estimators=150, xgb__reg_lambda=0.01, xgb__subsample=0.9)

In [4]:
xgb_experiments_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, xgb__colsample_bylevel=0.9,
       xgb__colsample_bytree=1.0, xgb__learning_rate=0.1, xgb__max_depth=1,
       xgb__n_estimators=150, xgb__reg_lambda=0.01, xgb__subsample=0.9)

In [24]:
pd.set_option('display.max_columns', 200)

experiments_df

standard_scaler__custom_values___0  standard_scaler__custom_values___1  \
0                                False                               False   
81                               False                               False   
32                                True                                True   
66                                True                                True   
34                                True                               False   
36                                True                                True   
37                                True                               False   
38                                True                               False   
82                               False                                True   
40                               False                                True   
41                                True                               False   
42                                True                                True   
43                               False                                True   
47                                True                               False   
28                                True                               False   
48                               False                                True   
77                                True                                True   
51                                True                               False   
52                               False                               False   
54                               False                                True   
55                                True                               False   
74                                True                               False   
73                               False                               False   
71                               False                                True   
70                               False                               False   
63                               False                               False   
85                               False                                True   
67                                True                                True   
13                               False                                True   
9                                False                                True   
..                                 ...                                 ...   
7                                 True                               False   
8                                 True                                True   
10                               False                               False   
11                                True                                True   
18                               False                               False   
20                                True                               False   
21                                True                               False   
22                               False                               False   
23                               False                                True   
27                                True                               False   
29                                True                                True   
30                                True                               False   
31                               False                                True   
35                                True                               False   
64                                True                               False   
39                                True                               False   
44                                True                                True   
45                               False                                True   
46                               False                                True   
49                        

In [12]:
# Transformation to polarized groups of shap values
polarized_df = pandas_shap_df.copy()
for i in range(0, len(pandas_shap_df.index)):
    for j in range(0, len(pandas_shap_df.columns)):
        if not experiments_df.iloc[i, j]:
            polarized_df.iloc[i, j] = -1 * pandas_shap_df.iloc[i, j]
polarized_df

standard_scaler__custom_values___0  standard_scaler__custom_values___1  \
0                                  0.0                                -0.0   
1                                  0.0                                -0.0   
2                                 -0.0                                 0.0   
3                                  0.0                                -0.0   
4                                 -0.0                                 0.0   
5                                 -0.0                                 0.0   
6                                 -0.0                                 0.0   
7                                 -0.0                                 0.0   
8                                  0.0                                 0.0   
9                                 -0.0                                -0.0   
10                                -0.0                                -0.0   
11                                -0.0                                -0.0   
12                                 0.0                                 0.0   
13                                -0.0                                 0.0   
14                                -0.0                                 0.0   
15                                -0.0                                -0.0   
16                                 0.0                                 0.0   
17                                 0.0                                -0.0   
18                                 0.0                                 0.0   
19                                -0.0                                 0.0   
20                                 0.0                                 0.0   
21                                 0.0                                -0.0   
22                                -0.0                                -0.0   
23                                 0.0                                 0.0   
24                                -0.0                                -0.0   
25                                -0.0                                 0.0   
26                                -0.0                                -0.0   
27                                -0.0                                 0.0   
28                                -0.0                                -0.0   
29                                -0.0                                 0.0   
..                                 ...                                 ...   
70                                -0.0                                 0.0   
71                                -0.0                                 0.0   
72                                 0.0                                -0.0   
73                                 0.0                                 0.0   
74                                 0.0                                -0.0   
75                                -0.0                                 0.0   
76                                 0.0                                -0.0   
77                                -0.0                                -0.0   
78                                -0.0                                -0.0   
79                                 0.0                                -0.0   
80                                -0.0                                 0.0   
81                                 0.0                                 0.0   
82                                -0.0                                 0.0   
83                                 0.0                                -0.0   
84                                 0.0                                -0.0   
85                                 0.0                                -0.0   
86                                -0.0                                 0.0   
87                                -0.0                                -0.0   
88                                -0.0                                -0.0   
89                        

In [13]:
# Certainly, I set a feature to True for large positive values
# Also, I set a feature to False for large negative values
# Otherwise, it is set to True or False
polarized_shap_result = polarized_df.sum()
polarized_shap_result.sort_values()

min_max_scaler__custom_values___4     -0.140224
standard_scaler__custom_values___0     0.000000
min_max_scaler__custom_values___8      0.000000
min_max_scaler__custom_values___9      0.000000
min_max_scaler__custom_values___10     0.000000
min_max_scaler__custom_values___11     0.000000
min_max_scaler__custom_values___12     0.000000
binarizer__custom_values___0           0.000000
binarizer__custom_values___1           0.000000
binarizer__custom_values___2           0.000000
binarizer__custom_values___4           0.000000
binarizer__custom_values___5           0.000000
binarizer__custom_values___6           0.000000
binarizer__custom_values___7           0.000000
binarizer__custom_values___8           0.000000
binarizer__custom_values___9           0.000000
binarizer__custom_values___10          0.000000
binarizer__custom_values___3           0.000000
min_max_scaler__custom_values___7      0.000000
min_max_scaler__custom_values___6      0.000000
min_max_scaler__custom_values___5      0

In [14]:
# Splits positive and negative

positive_fields = polarized_shap_result[polarized_shap_result > 0]
positive_fields = positive_fields / positive_fields.sum()
negative_fields = polarized_shap_result[polarized_shap_result < 0]
negative_fields = negative_fields / negative_fields.sum()
positive_fields = positive_fields[positive_fields > 0.05]
negative_fields = negative_fields[negative_fields > 0.05]

# Each iteration, find anything above % number

In [15]:
print(positive_fields, '\n', negative_fields)

# Splits positive and negative

# Each iteration, find anything above 5% and either remove a low value or remove a high value from their options

# Continue until 0 things were removed (0 will be removed if one option for each)

# When there are X choices yet

# Try appending experiments vs continue to use the same results for analysis ; keep together for now for review

# When there were 3

Series([], dtype: float32) 
 min_max_scaler__custom_values___4    1.0
dtype: float32


In [16]:
# just weighting based on feature length etc

In [17]:
for key in positive_fields.keys():
    choices = param_distributions[
        'standard_scaler__column_indices_to_replace'][2]
    if len(choices) > 1:
        param_distributions['standard_scaler__column_indices_to_replace'][
            2] = choices[:-1]

for key in negative_fields.keys():
    choices = param_distributions[key][2]
    if len(choices) > 1:
        param_distributions['standard_scaler__column_indices_to_replace'][
            2] = choices[1:]

KeyError: 'min_max_scaler__custom_values___4'

In [18]:
param_distributions['min_max_scaler__column_indices_to_replace']

KeyError: 'min_max_scaler__column_indices_to_replace'

In [19]:
# always consider all features
#

In [20]:
# featuers to consider,
# num of features
# Zeroes;
# could
# default distribution ()
# weighting by feature towards up or down

In [ ]:
for key in negative_fields.keys():
    print(key.split("___")[0])

In [ ]:
a = [1, 2, 3, 4]

In [ ]:
a[1:]

In [ ]:
a[:-1]